# Downloading Clips from the MusicCaps Dataset

In this notebook, we see how you can use `yt-dlp` to download clips from the [MusicCaps](https://huggingface.co/datasets/google/MusicCaps) dataset from Google. The MusicCaps dataset contains music and their associated text captions. You could use a dataset like this to train a text-to-audio generation model 😉.

Once we've downloaded the clips, we'll explore them using a [Gradio](https://gradio.app/) interface.

If you like this notebook:

  - consider giving the [repo](https://github.com/nateraw/download-musiccaps-dataset) a star ⭐️
  - consider following me on Github [@nateraw](https://github.com/nateraw)

In [1]:
%%capture
! pip install datasets[audio] yt-dlp

# For the interactive interface we'll need gradio
! pip install gradio

In [2]:
import subprocess
import os
from pathlib import Path

from datasets import load_dataset, Audio


def download_clip(
    video_identifier,
    output_filename,
    start_time,
    end_time,
    tmp_dir='/tmp/musiccaps',
    num_attempts=5,
    url_base='https://www.youtube.com/watch?v='
):
    status = False

    command = f"""
        yt-dlp --quiet --no-warnings -x --audio-format wav -f bestaudio -o "{output_filename}" --download-sections "*{start_time}-{end_time}" {url_base}{video_identifier}
    """.strip()

    attempts = 0
    while True:
        try:
            output = subprocess.check_output(command, shell=True,
                                                stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as err:
            attempts += 1
            if attempts == num_attempts:
                return status, err.output
        else:
            break

    # Check if the video was successfully saved.
    status = os.path.exists(output_filename)
    return status, 'Downloaded'


def main(
    data_dir: str,
    sampling_rate: int = 44100,
    limit: int = None,
    num_proc: int = 1,
    writer_batch_size: int = 1000,
):
    """
    Download the clips within the MusicCaps dataset from YouTube.
    Args:
        data_dir: Directory to save the clips to.
        sampling_rate: Sampling rate of the audio clips.
        limit: Limit the number of examples to download.
        num_proc: Number of processes to use for downloading.
        writer_batch_size: Batch size for writing the dataset. This is per process.
    """

    ds = load_dataset('google/MusicCaps', split='train')
    if limit is not None:
        print(f"Limiting to {limit} examples")
        ds = ds.select(range(limit))

    data_dir = Path(data_dir)
    data_dir.mkdir(exist_ok=True, parents=True)

    def process(example):
        outfile_path = str(data_dir / f"{example['ytid']}.wav")
        status = True
        if not os.path.exists(outfile_path):
            status = False
            status, log = download_clip(
                example['ytid'],
                outfile_path,
                example['start_s'],
                example['end_s'],
            )

        example['audio'] = outfile_path
        example['download_status'] = status
        return example

    return ds.map(
        process,
        num_proc=num_proc,
        writer_batch_size=writer_batch_size,
        keep_in_memory=False
    ).cast_column('audio', Audio(sampling_rate=sampling_rate))

## Load the Dataset

Here we are limiting to the first 32 examples. Since Colab is constrained to 2 cores, downloading the whole dataset here would take hours.

When running this on your own machine:
  - you can set `limit=None` to download + process the full dataset. Feel free to do that here in Colab, it'll just take a long time.
  - you should increase the `num_proc`, which will speed things up substantially
  - If you run out of memory, try reducing the `writer_batch_size`, as by default, it will keep 1000 examples in memory *per worker*.

In [4]:
ds = main('./music_data', num_proc=2, limit=32)

Limiting to 32 examples


Let's explore the samples using a quick Gradio Interface 🤗

In [5]:
import gradio as gr

def get_example(idx):
    ex = ds[idx]
    return ex['audio']['path'], ex['caption']

gr.Interface(
    get_example,
    inputs=gr.Slider(0, len(ds) - 1, value=0, step=1),
    outputs=['audio', 'textarea'],
    live=True
).launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
